<font color = "purple"> First, retrieve a list of available `ObjectID`s according to Gallery number.

In [6]:
import requests
import pandas as pd

#a function which takes a PMA API access token and list of gallery numbers  
#and saves to disk a csv file of object ids of objects in the given galleries. 

def get_objectids(token, galleries) : 
    frames = []
    url = "https://hackathon.philamuseum.org/api/v0/collection/object/location"

    for name in galleries : 
        param = {"api_token":token, "name":name} 
        r = requests.get(url, param) 
    
        if r.status_code == 200: 
            data = r.json()
            ids = data["ObjectIDs"]
            num_of_objects = len(ids) 
            loc = [name for i in range(num_of_objects)]
        
            d = {"Gallery Name": loc, "ObjectID": ids} 
            df = pd.DataFrame(data = d) 
        
            frames.append(df) 
            
    objectids = pd.concat(frames)
    objectids.to_csv("objectids.csv", index = False)     

In [12]:
#create a list of ALL galleries in the PMA and retrieve objectids for this list. 

g1 = [i for i in range(100,125)]
g2 = [i for i in range(150,189)] 
g3 = ["165a", "167a"] 

g4 = [i for i in range(200,300)] 
g5 = ["213a", "233a", "240a", "276a", "277a", "290a"] 

gallery_numbers = g1 + g2 + g3 + g4 + g5

get_objectids(token, gallery_numbers) 

<font color = "purple"> Having gathered a list of `ObjectID`s, retrieve info about the objects. 

In [7]:
import pandas as pd
import numpy as np
import requests 

#a function which takes a PMA API access token and list of object ids   
#and saves to disk a csv file of features for each object id

def get_objectinfo(token, ids) : 

    #parameters to be passed into the api request, and then making the api request
    p = {"api_token":token, "objectId":ids[0]} 
    req = requests.get(url, p) 

    #write the first row of the DataFrame, if the status code is successful. 
    #then extract relevant features from the json output, and write this as the next row of the DataFrame.
    if req.status_code == 200 : 
        js = req.json() 
        obj_features = [[js["ObjectID"], js["Classification"],js["Style"], js["Dated"], js["SocialTags"] ] ]
        df = pd.DataFrame(obj_features, columns =["ObjectID", "Classification", "Style", "Dated", "SocialTags"]) 
        df.to_csv("objectinfo.csv", index = False)        
        
    #do the same as above, now looping over all the remaining objects.
    for ind in range(len(ids)-1): 
        p = {"api_token":token, "objectId":ids[ind+1]} 
        req = requests.get(url, p) 
    
        if req.status_code == 200 : 
            js = req.json()
            obj_features = [[js["ObjectID"], js["Classification"],js["Style"], js["Dated"], js["SocialTags"] ] ]
            df = pd.DataFrame(obj_features, columns =["ObjectID", "Classification", "Style", "Dated", "SocialTags"]) 
            df.to_csv("objectinfo.csv", mode = 'a', index = False, header = False)        

    #this slows the computation, since you can make 500 requests per minute.
        if ind%498 == 0 : 
            time.sleep(60)